In [12]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import RegressionEvaluator


https://www.kaggle.com/datasets/tanay001/nseindia-futures-options-daily/data

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.config('spark.driver.memory', '32g').appName('Future and Option Data Analysis').getOrCreate()


/home/lplab/anaconda3/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


In [4]:

# Load your dataset
data = spark.read.csv("fobhav.csv", header=True, inferSchema=True)

# Filter rows where INSTRUMENT is FUTIDX
futures_data = data.filter(data['INSTRUMENT'] == 'FUTIDX')

# Show the filtered data
futures_data.show()




+----------+---------+-----------+---------+----------+--------+--------+--------+--------+---------+---------+----------+--------+---------+-----------+-----------+
|INSTRUMENT|   SYMBOL|  EXPIRY_DT|STRIKE_PR|OPTION_TYP|    OPEN|    HIGH|     LOW|   CLOSE|SETTLE_PR|CONTRACTS|VAL_INLAKH|OPEN_INT|CHG_IN_OI|  TIMESTAMP|Unnamed: 15|
+----------+---------+-----------+---------+----------+--------+--------+--------+--------+---------+---------+----------+--------+---------+-----------+-----------+
|    FUTIDX|BANKNIFTY|24-Sep-2020|      0.0|        XX|24900.05| 25270.0| 23371.0| 23743.0|  23743.0|   504222|3051593.27| 1432850|   -54500|31-AUG-2020|       null|
|    FUTIDX|BANKNIFTY|29-Oct-2020|      0.0|        XX|24937.55| 25315.9| 23391.9| 23770.1|  23770.1|    10630|  64526.57|   91375|    19325|31-AUG-2020|       null|
|    FUTIDX|BANKNIFTY|26-Nov-2020|      0.0|        XX|24989.95|25280.75|23437.15| 23766.2|  23766.2|     1171|   7088.68|   14400|     8400|31-AUG-2020|       null|
|   

In [20]:
futures_data.count()

94010

In [25]:
from pyspark.sql.functions import col

# Assuming futures_data is your DataFrame
futures_data = futures_data.withColumn("SETTLE_PR", col("SETTLE_PR").cast("double"))


##### Now clean the FUTIDX data frame where the option type is xx

In [6]:
futures_data.show(vertical=True)

-RECORD 0------------------
 INSTRUMENT  | FUTIDX      
 SYMBOL      | BANKNIFTY   
 EXPIRY_DT   | 24-Sep-2020 
 STRIKE_PR   | 0.0         
 OPTION_TYP  | XX          
 OPEN        | 24900.05    
 HIGH        | 25270.0     
 LOW         | 23371.0     
 CLOSE       | 23743.0     
 SETTLE_PR   | 23743.0     
 CONTRACTS   | 504222      
 VAL_INLAKH  | 3051593.27  
 OPEN_INT    | 1432850     
 CHG_IN_OI   | -54500      
 TIMESTAMP   | 31-AUG-2020 
 Unnamed: 15 | null        
-RECORD 1------------------
 INSTRUMENT  | FUTIDX      
 SYMBOL      | BANKNIFTY   
 EXPIRY_DT   | 29-Oct-2020 
 STRIKE_PR   | 0.0         
 OPTION_TYP  | XX          
 OPEN        | 24937.55    
 HIGH        | 25315.9     
 LOW         | 23391.9     
 CLOSE       | 23770.1     
 SETTLE_PR   | 23770.1     
 CONTRACTS   | 10630       
 VAL_INLAKH  | 64526.57    
 OPEN_INT    | 91375       
 CHG_IN_OI   | 19325       
 TIMESTAMP   | 31-AUG-2020 
 Unnamed: 15 | null        
-RECORD 2------------------
 INSTRUMENT  | FUTID

In [26]:
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler


# Convert string columns to numeric types
futures_data = futures_data.withColumn('OPEN', futures_data['OPEN'].cast('double'))
futures_data = futures_data.withColumn('HIGH', futures_data['HIGH'].cast('double'))
futures_data = futures_data.withColumn('LOW', futures_data['LOW'].cast('double'))
futures_data = futures_data.withColumn('CLOSE', futures_data['CLOSE'].cast('double'))
futures_data = futures_data.withColumn('SETTLE_PR', futures_data['SETTLE_PR'].cast('double')) # Assuming SETTLE_PR is the label column

# Now you can proceed with the VectorAssembler and model training
feature_columns = ['OPEN', 'HIGH', 'LOW', 'CLOSE']
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data = assembler.transform(futures_data)

# Define the neural network architecture
layers = [len(feature_columns), 64, 32, 1]  # Input size, hidden layers, output size
mlp = MultilayerPerceptronClassifier(layers=layers, seed=1, featuresCol="features", labelCol="SETTLE_PR") # Use SETTLE_PR as labelCol

# Split data into training and test sets
(training_data, test_data) = data.randomSplit([0.8, 0.2])

# Train the neural network model
model = mlp.fit(training_data)

# Make predictions on the test data
predictions = model.transform(test_data)


Py4JJavaError: An error occurred while calling o540.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 15 in stage 24.0 failed 1 times, most recent failure: Lost task 15.0 in stage 24.0 (TID 601) (lpcp-01 executor driver): java.lang.RuntimeException: Labels MUST be in [0, 1), but got 24955.2
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.project_doConsume_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:760)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.Iter

In [9]:
# Evaluate the model
evaluator = RegressionEvaluator(labelCol="OPTION_PRICE", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE):", rmse)

In [24]:
futures_data.printSchema()

root
 |-- INSTRUMENT: string (nullable = true)
 |-- SYMBOL: string (nullable = true)
 |-- EXPIRY_DT: string (nullable = true)
 |-- STRIKE_PR: string (nullable = true)
 |-- OPTION_TYP: string (nullable = true)
 |-- OPEN: double (nullable = true)
 |-- HIGH: double (nullable = true)
 |-- LOW: double (nullable = true)
 |-- CLOSE: double (nullable = true)
 |-- SETTLE_PR: double (nullable = true)
 |-- CONTRACTS: string (nullable = true)
 |-- VAL_INLAKH: string (nullable = true)
 |-- OPEN_INT: string (nullable = true)
 |-- CHG_IN_OI: string (nullable = true)
 |-- TIMESTAMP: string (nullable = true)
 |-- Unnamed: 15: string (nullable = true)

